# 🍎 Fruit Freshness: GAN vs VAE Karşılaştırması (Google Colab Versiyonu)

Bu notebook, Fruit Freshness Classification veri seti üzerinde DCGAN ve VAE modellerinin karşılaştırmalı analizini içerir.
Tüm kodlar (veri indirme, modeller, eğitim) bu notebook içinde tanımlanmıştır.

## İçerik
1. Kurulum ve Kütüphaneler
2. Veri Seti İndirme (kagglehub)
3. Yardımcı Fonksiyonlar
4. Model Mimarileri (VAE & DCGAN)
5. VAE Eğitimi
6. DCGAN Eğitimi
7. Sonuçlar ve Karşılaştırma

## 1. Kurulum ve Kütüphaneler

In [ ]:
!pip install kagglehub -q

import os
import shutil
import random
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms
from torchvision.utils import make_grid

# Seed ayarlama (Tekrarlanabilirlik için)
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Device ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

## 2. Veri Seti İndirme

In [ ]:
import kagglehub

print("Veri seti indiriliyor (kagglehub ile)...")
try:
    # Veriyi indir
    cache_path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")
    print(f"İndirilen yol: {cache_path}")
    
    # Veri setini düzenle
    # Veri setinin iç yapısını kontrol edip doğru train klasörünü buluyoruz
    data_root = Path(cache_path)
    train_dir = None
    
    # Olası yolları kontrol et
    possible_paths = [
        data_root / "dataset" / "train",
        data_root / "train",
        data_root
    ]
    
    for path in possible_paths:
        if path.exists() and any(path.iterdir()):
            # İçinde freshapples vb. klasörler var mı diye bak
            if any((path / cls).exists() for cls in ['freshapples', 'rottenapples']):
                train_dir = path
                break
    
    if train_dir:
        print(f"Eğitim verisi bulundu: {train_dir}")
    else:
        raise FileNotFoundError("Eğitim klasörü (train) bulunamadı!")
        
except Exception as e:
    print(f"Hata oluştu: {e}")

## 3. Yardımcı Fonksiyonlar (Dataloader & Görselleştirme)

In [ ]:
# Görüntü dönüşümleri
def get_transforms(image_size=64):
    return transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # [-1, 1] aralığına getirir
    ])

# Dataloader oluşturucu
def create_dataloader(root_dir, batch_size=32, image_size=64, val_split=0.1):
    transform = get_transforms(image_size)
    
    dataset = datasets.ImageFolder(root=root_dir, transform=transform)
    print(f"Toplam görüntü sayısı: {len(dataset)}")
    print(f"Sınıflar: {dataset.classes}")
    
    # Train/Val split
    val_size = int(len(dataset) * val_split)
    train_size = len(dataset) - val_size
    
    train_dataset, val_dataset = random_split(
        dataset, [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    
    return train_loader, val_loader

# Görselleştirme fonksiyonları
def denormalize(tensor):
    return (tensor + 1) / 2

def show_images(images, nrow=8, title=None):
    images = denormalize(images.cpu())
    grid = make_grid(images, nrow=nrow, padding=2)
    plt.figure(figsize=(10, 10))
    plt.imshow(grid.permute(1, 2, 0))
    plt.axis('off')
    if title:
        plt.title(title)
    plt.show()

# Veriyi yükle
train_loader, val_loader = create_dataloader(train_dir, batch_size=64, image_size=64)

# Örnek bir batch göster
sample_images, _ = next(iter(train_loader))
show_images(sample_images[:16], nrow=4, title="Eğitim Verisinden Örnekler")

## 4. Model Mimarileri

In [ ]:
# --- VAE Modeli ---
class VAE(nn.Module):
    def __init__(self, latent_dim=128):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1), nn.BatchNorm2d(256), nn.LeakyReLU(0.2),
        )
        
        self.fc_mu = nn.Linear(256*4*4, latent_dim)
        self.fc_logvar = nn.Linear(256*4*4, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, 256*4*4)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1), nn.Tanh()
        )
        
    def encode(self, x):
        h = self.encoder(x)
        h = h.view(h.size(0), -1)
        return self.fc_mu(h), self.fc_logvar(h)
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
        
    def decode(self, z):
        h = self.fc_decode(z)
        h = h.view(h.size(0), 256, 4, 4)
        return self.decoder(h)
        
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def vae_loss_fn(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss, recon_loss, kl_loss

# --- DCGAN Modelleri ---
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
    def __init__(self, noise_dim=100):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(noise_dim, 512, 4, 1, 0, bias=False), nn.BatchNorm2d(512), nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False), nn.Tanh()
        )
        self.apply(weights_init)

    def forward(self, z):
        return self.main(z)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), nn.BatchNorm2d(128), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.BatchNorm2d(256), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False), nn.BatchNorm2d(512), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False), nn.Sigmoid()
        )
        self.apply(weights_init)

    def forward(self, x):
        return self.main(x)

## 5. VAE Eğitimi

In [ ]:
# VAE Ayarları
vae = VAE(latent_dim=128).to(device)
vae_optimizer = optim.Adam(vae.parameters(), lr=1e-3)
vae_epochs = 20  # Demo için kısa tutuldu, artırılabilir

print("VAE eğitimi başlıyor...")
vae_losses = []

for epoch in range(vae_epochs):
    vae.train()
    total_loss = 0
    for images, _ in tqdm(train_loader, desc=f"VAE Epoch {epoch+1}/{vae_epochs}", leave=False):
        images = images.to(device)
        
        recon_images, mu, logvar = vae(images)
        loss, _, _ = vae_loss_fn(recon_images, images, mu, logvar)
        
        vae_optimizer.zero_grad()
        loss.backward()
        vae_optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader.dataset)
    vae_losses.append(avg_loss)
    print(f"Epoch {epoch+1}: Loss {avg_loss:.4f}")
    
    # Her 5 epochta bir örnek göster
    if (epoch + 1) % 5 == 0:
        vae.eval()
        with torch.no_grad():
            z = torch.randn(16, 128).to(device)
            samples = vae.decode(z)
            show_images(samples, nrow=8, title=f"VAE Samples Epoch {epoch+1}")

## 6. DCGAN Eğitimi

In [ ]:
# DCGAN Ayarları
netG = Generator(noise_dim=100).to(device)
netD = Discriminator().to(device)

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(64, 100, 1, 1, device=device)
gan_epochs = 20

print("DCGAN eğitimi başlıyor...")
G_losses = []
D_losses = []

for epoch in range(gan_epochs):
    for i, (data, _) in enumerate(tqdm(train_loader, desc=f"GAN Epoch {epoch+1}/{gan_epochs}", leave=False)):
        # 1. Discriminator Eğitimi: log(D(x)) + log(1 - D(G(z)))
        netD.zero_grad()
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 1., dtype=torch.float, device=device)
        
        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        
        noise = torch.randn(b_size, 100, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(0.)
        
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        optimizerD.step()
        
        # 2. Generator Eğitimi: log(D(G(z)))
        netG.zero_grad()
        label.fill_(1.)  # Generator için hedef: Discriminator'ı kandırmak
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()
        
        G_losses.append(errG.item())
        D_losses.append(errD_real.item() + errD_fake.item())

    print(f"Epoch {epoch+1}: Loss_D: {D_losses[-1]:.4f}, Loss_G: {G_losses[-1]:.4f}")
    
    if (epoch + 1) % 5 == 0:
        with torch.no_grad():
            fake = netG(fixed_noise).detach().cpu()
            show_images(fake, nrow=8, title=f"DCGAN Samples Epoch {epoch+1}")

## 7. Sonuçlar ve Karşılaştırma

In [ ]:
# Loss Grafikleri
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(vae_losses, label="VAE Loss")
plt.title("VAE Training Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(G_losses, label="Generator")
plt.plot(D_losses, label="Discriminator")
plt.title("DCGAN Training Losses")
plt.legend()
plt.show()

# Yan Yana Karşılaştırma
vae.eval()
netG.eval()

with torch.no_grad():
    # VAE'den örnekler
    z_vae = torch.randn(32, 128).to(device)
    vae_samples = vae.decode(z_vae)
    
    # GAN'dan örnekler
    z_gan = torch.randn(32, 100, 1, 1).to(device)
    gan_samples = netG(z_gan)

print("VAE Sonuçları:")
show_images(vae_samples, nrow=8)

print("DCGAN Sonuçları:")
show_images(gan_samples, nrow=8)